<a href="https://colab.research.google.com/github/sunil-karki/NLP-Question_Classification/blob/master/NLP-DisasterTweets_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sunilkarki520","key":"8368d7fdc07a580b54aeb9a6df65cda4"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 46.5MB/s]
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 58.0MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 59.7MB/s]


In [4]:
!ls

kaggle.json  sample_data  sample_submission.csv  test.csv  train.csv


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate

from keras.optimizers import Adam
from keras import backend as K      ######
from keras.models import Model
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [6]:

train_df = pd.read_csv('train.csv') 
test_df = pd.read_csv('test.csv') 
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

Train shape :  (7613, 5)
Test shape :  (3263, 4)


In [7]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [8]:
test_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [0]:

train_data = train_df 
train_data = train_data.replace(np.nan, '', regex=True)

train_data['text'] = train_data['text'] + ' ' + train_data['location'] + ' ' + train_data['keyword']
del train_data['keyword']
del train_data['location']
#train_data.iloc[0]
a = train_data.iloc[0][['text']].values
#print(a)

test_data = test_df 
test_data = test_data.replace(np.nan, '', regex=True)

test_data['text'] = test_data['text'] + ' ' + test_data['location'] + ' ' + test_data['keyword']

In [37]:
train_data

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...
7608,10869,Two giant cranes holding a bridge collapse int...,1
7609,10870,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,Police investigating after an e-bike collided ...,1


In [0]:
## some config values 
embed_size = 300 # how big is each word vector
max_features = 95000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 70 # max number of words in a question to use

## ValueError: Layer weight shape (95000, 300) not compatible with provided weight shape (24568, 300)
max_features = 24568

In [39]:
maxlen = max([len(data) for data in train_data['text'].values])   ##maximum number of words in column text
print(maxlen)    ##195

195


In [0]:
## split to train and val
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=2020)
train_data, traintest_data = train_test_split(train_data, test_size=0.2, random_state=2019) 
 
## fill up the missing values
train_X = train_data['text'].values
val_X = val_data['text'].values
test_X = test_data['text'].values
traintest_X = traintest_data['text'].values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)
traintest_X = tokenizer.texts_to_sequences(traintest_X)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)
traintest_X = pad_sequences(traintest_X, maxlen=maxlen)

## Get the target values
train_y = train_data['target'].values   #train_y is an array i.e. array([1, 0, 1, ..., 1, 1, 1])
val_y = val_data['target'].values
traintest_y = traintest_data['target'].values  
    
#shuffling the data
np.random.seed(2018)
trn_idx = np.random.permutation(len(train_X))
val_idx = np.random.permutation(len(val_X))
trntst_idx = np.random.permutation(len(traintest_X))

train_X = train_X[trn_idx]  #train_X is an array i,e. array([1653, 2280, 1170, ..., 6370, 2662, 1274])
val_X = val_X[val_idx]
train_y = train_y[trn_idx]
val_y = val_y[val_idx]    
traintest_X = traintest_X[trntst_idx] 
traintest_y = traintest_y[trntst_idx]

In [41]:
word_index = tokenizer.word_index
print(word_index)

{'co': 1, 't': 2, 'http': 3, 'the': 4, 'a': 5, 'in': 6, 'to': 7, 'of': 8, 'and': 9, 'i': 10, 'is': 11, 'for': 12, 'on': 13, 'you': 14, 'my': 15, 'with': 16, 'that': 17, 'it': 18, 'at': 19, 'by': 20, 'this': 21, 'new': 22, 'are': 23, 'https': 24, 'be': 25, 'from': 26, 'have': 27, 'was': 28, 'like': 29, 'amp': 30, 'fire': 31, 'me': 32, 'as': 33, 'just': 34, 'up': 35, 'not': 36, 'but': 37, 'so': 38, 'your': 39, 'usa': 40, 'all': 41, 'out': 42, 'emergency': 43, 'has': 44, 'after': 45, 'an': 46, "i'm": 47, 'body': 48, 'no': 49, 'if': 50, 'now': 51, 'will': 52, 'we': 53, 'via': 54, '2': 55, 'or': 56, 'when': 57, 'he': 58, 'get': 59, 'about': 60, 'suicide': 61, 'more': 62, 'how': 63, 'they': 64, 'people': 65, 'who': 66, 'california': 67, "it's": 68, 'one': 69, 'disaster': 70, 'been': 71, 'ca': 72, 'police': 73, 'nuclear': 74, 'buildings': 75, 'do': 76, 'what': 77, "'": 78, 'over': 79, "don't": 80, 'into': 81, 'burning': 82, 'news': 83, 'can': 84, 'u': 85, 'york': 86, 'crash': 87, 'storm': 88,

In [42]:
train_X

array([[    0,     0,     0, ..., 13441,   633,   204],
       [    0,     0,     0, ...,   288,    99,   768],
       [    0,     0,     0, ...,  1159,   355,   265],
       ...,
       [    0,     0,     0, ...,    91,   633,   186],
       [    0,     0,     0, ...,  4908,   296,   326],
       [    0,     0,     0, ...,     8,   679,   375]], dtype=int32)

In [16]:
!kaggle competitions download -c quora-insincere-questions-classification

100% 5.95G/5.96G [01:22<00:00, 53.7MB/s]
100% 5.96G/5.96G [01:22<00:00, 77.4MB/s]
 57% 9.00M/15.8M [00:00<00:00, 34.6MB/s]
100% 15.8M/15.8M [00:00<00:00, 45.6MB/s]
  0% 0.00/4.09M [00:00<?, ?B/s]
100% 4.09M/4.09M [00:00<00:00, 37.6MB/s]
 75% 41.0M/54.9M [00:00<00:00, 54.5MB/s]
100% 54.9M/54.9M [00:00<00:00, 86.6MB/s]


In [17]:
!ls

embeddings.zip	sample_submission.csv	   test.csv.zip
kaggle.json	sample_submission.csv.zip  train.csv
sample_data	test.csv		   train.csv.zip


In [18]:
!unzip embeddings.zip

Archive:  embeddings.zip
   creating: GoogleNews-vectors-negative300/
   creating: glove.840B.300d/
   creating: paragram_300_sl999/
   creating: wiki-news-300d-1M/
  inflating: glove.840B.300d/glove.840B.300d.txt  
  inflating: GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin  
  inflating: wiki-news-300d-1M/wiki-news-300d-1M.vec  
  inflating: paragram_300_sl999/README.txt  
  inflating: paragram_300_sl999/paragram_300_sl999.txt  


In [19]:
!ls

embeddings.zip			sample_data		   train.csv
glove.840B.300d			sample_submission.csv	   train.csv.zip
GoogleNews-vectors-negative300	sample_submission.csv.zip  wiki-news-300d-1M
kaggle.json			test.csv
paragram_300_sl999		test.csv.zip


In [0]:
def load_glove(word_index):
    # EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    EMBEDDING_FILE = 'glove.840B.300d/glove.840B.300d.txt'
    ## This file contains content as following
    #Glass -0.41236 0.35532 -0.076236 0.43268 0.14769 -1.1381 -0.30427 0.37039 -0.3804 0.029443 -0.081454 0.15791 -0.32558 -0.35445 0.11918 -0.13342 -0.087387 1.2125 -0.048223  .....
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 

In [44]:
embedding_matrix_1 = load_glove(word_index)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


IndexError: ignored

In [0]:
# https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [0]:
def model_gru_atten_3(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    #x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    #x = Bidirectional(CuDNNGRU(100, return_sequences=True))(x)
    #x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    x = Bidirectional(GRU(100, return_sequences=True))(x)
    x = Bidirectional(GRU(64, return_sequences=True))(x)
    
    x = Attention(maxlen)(x)        ## Finding on this
    
    x = Dense(1, activation="sigmoid")(x)
    #x = Dense(1, activation="softmax")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [0]:
'''
def train_pred(model, epochs_param):
    model.fit(train_X, train_y, batch_size=512, epochs = epochs_param, validation_data=(val_X, val_y))
    pred_val_y = model.predict([val_X], batch_size=1024, verbose=1)
    pred_test_y = model.predict([test_X], batch_size=1024, verbose=1)
    return pred_val_y, pred_test_y, model
'''

In [0]:
from keras.models import load_model

def train_pred(model, epochs_param):
    try:
        model.fit(train_X, train_y, batch_size=512, epochs = epochs_param, validation_data=(val_X, val_y))
    except KeyboardInterrupt:
        filename = 'google3qla.h5'
        model.save(filename)
        model = load_model('google3qla.h5')

    pred_val_y = model.predict([val_X], batch_size=1024, verbose=1)
    pred_test_y = model.predict([test_X], batch_size=1024, verbose=1)
    return pred_val_y, pred_test_y, model

In [0]:
##  ValueError: Layer weight shape (95000, 300) not compatible with provided weight shape (24568, 300)

In [55]:
pred_val_y, pred_test_y, model = train_pred(model_gru_atten_3(embedding_matrix_1), epochs_param = 20)

Train on 4872 samples, validate on 1523 samples
Epoch 1/20
4872/4872 [==============================] - 80s 16ms/step - loss: 0.7845 - acc: 0.5250 - val_loss: 0.7208 - val_acc: 0.5575
Epoch 2/20
4872/4872 [==============================] - 71s 14ms/step - loss: 0.6826 - acc: 0.5665 - val_loss: 0.6912 - val_acc: 0.5575
Epoch 3/20
4872/4872 [==============================] - 70s 14ms/step - loss: 0.6706 - acc: 0.5784 - val_loss: 0.6661 - val_acc: 0.5877
Epoch 4/20
4872/4872 [==============================] - 70s 14ms/step - loss: 0.6587 - acc: 0.5870 - val_loss: 0.6614 - val_acc: 0.5804
Epoch 5/20
4872/4872 [==============================] - 70s 14ms/step - loss: 0.6476 - acc: 0.6119 - val_loss: 0.6420 - val_acc: 0.6093
Epoch 6/20
4872/4872 [==============================] - 70s 14ms/step - loss: 0.6332 - acc: 0.6355 - val_loss: 0.6218 - val_acc: 0.6592
Epoch 7/20
4872/4872 [==============================] - 70s 14ms/step - loss: 0.6138 - acc: 0.6624 - val_loss: 0.5957 - val_acc: 0.6940


In [56]:
pred_test_y

array([[0.10713273],
       [0.24778205],
       [0.55232704],
       ...,
       [0.99577063],
       [0.99415016],
       [0.05500104]], dtype=float32)

In [57]:
score = model.evaluate(traintest_X, traintest_y, verbose=0)
print('test loss, test acc:', score)

test loss, test acc: [0.775773223789259, 0.7389162561576355]


In [60]:
print(pred_val_y.shape)
print(test_X.shape)

(1523, 1)
(3263, 195)


In [59]:
pred_test_y_a = np.where( pred_test_y > 0.33 , 1, 0)

#pred_val_y_a = (pred_val_y > 0.33).astype(int)
print(pred_val_y_a.shape)

(8, 1)


In [64]:
pred = model.predict([traintest_X])
print(pred)

[[0.99807084]
 [0.03940597]
 [0.9826474 ]
 ...
 [0.01382792]
 [0.08185907]
 [0.9215751 ]]


In [83]:
pred_a = np.where( pred > 0.42 , 1, 0 )  #0.33   0.42
print(pred_a)

[[1]
 [0]
 [1]
 ...
 [0]
 [0]
 [1]]


In [0]:
a = pred_a
for i in range(0, len(pred_a)):
  #print(i)
  if pred_a[i] == traintest_y[i]:
    a[i] = 1
  else:
    a[i] = 0



In [85]:
acc = np.mean(a)*100
print(acc)

73.39901477832512


In [87]:
pred_test_y.shape

(3263, 1)

In [67]:
check = np.equal(pred_a, traintest_y)
#check_1 = np.mean(check, axis=0)
print(check)

[[ True  True False ...  True False False]
 [False False  True ... False  True  True]
 [ True  True False ...  True False False]
 ...
 [False False  True ... False  True  True]
 [False False  True ... False  True  True]
 [ True  True False ...  True False False]]


In [66]:
check = np.equal(pred_val_y_a, val_y)
#check = np.where( np.equal(pred_val_y_a, val_y) == True, 1, 0) 
#check_1 = np.mean(check, axis=0)
print(check)

[[False  True False ...  True False False]
 [False  True False ...  True False False]
 [ True False  True ... False  True  True]
 ...
 [ True False  True ... False  True  True]
 [False  True False ...  True False False]
 [False  True False ...  True False False]]


In [0]:
for i, predicted in enumerate(predictions):
    if predicted[0] > 0.25:
        print "bigger than 0.25"
        #assign i to class 1
    else:
        print "smaller than 0.25"
        #assign i to class 0

In [0]:
a = pred_test_y

In [0]:
b = a.astype(int)  #numpy
print(b)

<class 'numpy.ndarray'>


In [0]:
unique_elements, counts_elements = np.unique(b, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

[[   0]
 [3263]]
